In [72]:
import pandas as pd
import requests
import sqlite3
import re
from bs4 import BeautifulSoup

In [73]:
events = ["pro-tour-london", "us-national-championship-2025", "pro-tour-singapore-2025"]
database = "fab_world_tour.db"
draft_rounds_start = 6
draft_rounds_end = 11
rounds_total = 18

decklists_url = "https://fabtcg.com/en/coverage/{}/decklist/{}/"
results_url = "https://fabtcg.com/en/coverage/{}/results/{}/"

In [74]:
rounds = [number + 1 for number in range(rounds_total) if not draft_rounds_start <= number + 1 <= draft_rounds_end]

In [75]:
def get_pairings(url, event, rounds_to_be_considered):
    output = []

    for round in rounds_to_be_considered:
        page = requests.get(url.format(event, round))
        soup = BeautifulSoup(page.text, "html")

        player = soup.find_all("div", {"class":"tournament-coverage__player-hero-and-deck"})
        winner = soup.find_all("div", {"class":"tournament-coverage__result"})

        for idx, id in enumerate(player):
            table = int(idx / 2) + 1

            seat = (idx % 2) + 1

            gem_id = id.find("a", href = True)

            if gem_id:
                gem_id = re.search(r"\/(\d+)", gem_id["href"]).group(1)

            if re.search(r"(\d)", winner[table - 1].text):
                winning_seat = int(re.search(r"(\d)", winner[table - 1].text).group(1))
            else:
                print(f"Round: {round} Table: {table} had a draw")
                winning_seat = 0

            if winning_seat == 0:
                player_status = "Draw"
            elif winning_seat == seat:
                player_status = "Win"
            else:
                player_status = "Loss"

            record = [event, round, table, seat, gem_id, player_status]

            output.append(record)

    return output

def make_player_list(pairings):
    output = []

    for record in pairings:
        if record[4] not in output and record[4]:
            output.append(record[4])

    return output

def get_decklist(event, player_list, decklist_url):
    output = []

    for id in player_list:
        url = decklist_url.format(event, id)

        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html")

        if page.status_code == 200:
            output.append([data.text.strip() for data in soup.find_all("td")])
        else:
            output.append([id, "Unknown", None, None, None, "Unknown"])

    return output

def decklist_to_df(event, decklists):
    decklists = pd.DataFrame.from_records(decklists)
    decklists.index = [re.search(r"(\d+)", name).group(1) for name in decklists[0]]

    participants = decklists.iloc[:,0:5].copy()
    participants = participants.drop(2, axis=1)
    participants = participants.rename(columns={0:"Name", 1:"Event Date", 3:"Format", 4:"Hero"})
    participants["Event Name"] = event

    participants.index.rename("Gem ID", inplace=True) 

    decklists = decklists.drop([0,1,2,3,4], axis=1)

    decklists = pd.melt(decklists, ignore_index=False, value_name="import name")["import name"].to_frame()

    decklists[["Copies","Card"]] = decklists["import name"].str.split(" x ", expand=True)
    decklists = decklists.drop("import name", axis=1).dropna()

    decklists.index.rename("Gem ID", inplace=True)
    decklists["Event Name"] = event

    return participants, decklists

def df_to_sql(dataframe, sql_table_name, index=True):
    connection = sqlite3.connect(database)

    dataframe.to_sql(sql_table_name, connection, if_exists="append", index=index)

In [ ]:
for event in events:
    pairings = get_pairings(results_url, event, rounds)
    player_list = make_player_list(pairings)
    decklists = get_decklist(event, player_list, decklists_url)
    tournament = decklist_to_df(event, decklists)
    participant_df = tournament[0]
    decklist_df = tournament[1]
    pairings_df = pd.DataFrame.from_records(pairings, columns=["Event", "Round","Table","Seat","Gem ID","Outcome"])


    df_to_sql(participant_df, "participants")
    df_to_sql(decklist_df, "decklists")
    df_to_sql(pairings_df, "pairings", False)

In [83]:
# Tableau public doesn't like SQL connections, so until then I need to export to an xlsx.
cnxn = sqlite3.connect(database)

tournament_results = pd.read_sql("Select * From results", cnxn)
tournament_decks = pd.read_sql("Select * From tournament_lists", cnxn)
tournament_events = pd.read_sql("Select * From events", cnxn)

with pd.ExcelWriter("fab_world_tour.xlsx") as writer:
    tournament_results.to_excel(writer, sheet_name="results", index=False)
    tournament_decks.to_excel(writer, sheet_name="decks", index=False)
    tournament_events.to_excel(writer, sheet_name="events", index=False)